In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle

### GLOBAL CONFIG

In [57]:


PREPROCESS = False # Do a fresh preprocess
MAKE_NEW_EMBEDDING = True # If False, the stored one will be loaded
# EMB_MAX_WORDS = None
EMB_MAX_WORDS = 5000
RANDOM_SEED = 456

PREPROCESS_INPUT = './data/training.1600000.processed.noemoticon.csv'
PREPROCESS_OUTPUT = './data/preprocessed.csv'
GLOVE_FILE = './data/glove.6B.50d.txt'
EMB_PKL = './models/emb_layer.pkl'
MODEL_PKL = './models/model.pkl'


### Preprocess data and store it

In [58]:
from pre import preprocess

if PREPROCESS:
    preprocess(i=PREPROCESS_INPUT, o=PREPROCESS_OUTPUT, slice=10)

### Load preprocessed data

In [76]:
from parse import load_data
from sklearn.model_selection import train_test_split

data = load_data(PREPROCESS_OUTPUT)
# TODO: may need to be done after train_test_split (test it)
X = data['tweet']
y = data['target'].apply(lambda y: tf.keras.utils.to_categorical(y, 2))

X, y = X.values, y.values

# X = [np.array(twt, dtype=str) for twt in X]

print(y[5], type())



# arrr = [a for a in X]
# print(arrr)
# print(type(X[0]))
# print(type(X), X.dtype, X[0].dtype)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=RANDOM_SEED)

[0. 1.]


### Embedding

In [60]:
from keras.preprocessing.text import Tokenizer
from emb import get_keras_embedding_layer


tknzr = Tokenizer(num_words=EMB_MAX_WORDS, lower=False, split=' ', oov_token="UNK")
if MAKE_NEW_EMBEDDING: 
    emb_layer = get_keras_embedding_layer(GLOVE_FILE, data['tweet'], tknzr)

    # save emb_layer
    with open(EMB_PKL, 'wb') as output:
        pickle.dump(emb_layer, output, pickle.HIGHEST_PROTOCOL)
else:
    with open(EMB_PKL, 'wb') as input:
        emb_layer = pickle.load(input)

Glove data loaded
Words not found 4


### Model

In [69]:
%load_ext autoreload
%autoreload 2
from TextClassifierModel import build_model_keras


# TODO: check correct arg-assignment for tokenizer
model = build_model_keras(tknzr, emb_layer)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
building model
init
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      multiple                  3550      
_________________________________________________________________
flatten_24 (Flatten)         (None, 5000)              0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 5000)              0         
_________________________________________________________________
dense_60 (Dense)             (None, 64)                320064    
_________________________________________________________________
dense_61 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_62 (Dense)             (None, 256)               33024     
___________

### Train

In [71]:
%load_ext autoreload
%autoreload 2
from evaluation import plot_history

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=60, epochs=30, verbose=1)
plot_history(history)

if SAVE_TRAINED_MODEL:
    with open(MODEL_PKL, 'wb') as output:
        pickle.dump(model, output, pickle.HIGHEST_PROTOCOL)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  2  4 19 20 21  4 22  6 23 24 25  5 26 27 28
  7 29 30 31]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

### Load trained model

In [ ]:
with open(MODEL_PKL, 'wb') as input:
    model = pickle.load(input)

### Evaluate production (forbidden during tuning)

In [ ]:
from evaluation import evaluate_model

evaluate_model(model, X_test, y_test)

### Explain prediction

In [ ]:
import eli5
from eli5.lime import TextExplainer

te = TextExplainer(random_state=42)
te.fit("I love candy. I like to be positive, be happy! What a lovely day", model.predict_proba)
te.show_prediction()